In [1]:
using Iterators
using DataStructures
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")

3-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              

In [2]:
using DataStructures

function Base.sum(acc::Accumulator)
    sum(values(acc.map))
end

function freq2prob{T,V<:Number}(acc::Accumulator{T,V})
    ret=Dict{T,Float64}()
    total = sum(acc)
    for (k,v) in acc
        ret[k]=v/total
    end
    ret
end

freq2prob (generic function with 1 method)

In [3]:
freq2prob(uu)

LoadError: LoadError: UndefVarError: uu not defined
while loading In[3], in expression starting on line 1

In [4]:
path = "../../Resources/corpora/atis2_text/"

"../../Resources/corpora/atis2_text/"

In [5]:
ss1=path*readdir(path)[10100
    ] |> readall

"List only the flights that leave before eight *A* M\n"

In [6]:
function clean(ss)
    @pipe (ss 
    |> replace(_, r"\[.*?\] ?","")  #Remove Nonword sounds
    |> replace(_, r"\<.*?\> ","")  #Remove Verbal Deltions
    |> replace(_, r"\*(.*?)\*",s"\1") #Remove mispronounciation marks
    |> replace(_,r"\:|\-\s\.\s\-", "") #remove intraword pauses
    |> replace(_,r"\w+\- ","") #remove stuttered words
    |> replace(_, r"[!\.,\?]","")        #Remove punctation as it is not used traditionally (see sro spec)
    
    |> replace(_, r"\s+",' ') #Remove repeated spaces
    #|> replace(_, r"([A-Z])\s([A-Z])", s"\1\2") #Merge len(2) abbrev
    |> replace(_, r".*[~\(\)\-\<\>#'].*","")#Remove everything if anything unfixable found
    |> lowercase
    |> strip 
    )
end


clean (generic function with 1 method)

In [7]:
function valid(ss)
    typeof(ss) <: ASCIIString  && length(ss)>0
end
    
corpus = @pipe readdir(path) |> filter!(fn -> splitext(fn)[2]==".sro", _) |> map(_) do fn
    try open(readall, path*fn) end
    end |> filter!(valid,_) |> map(clean,_) |> filter!(valid,_) |> map(s->split(s),_);
length(corpus)

13149

In [8]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

function collect_grams_stats(sentences)
    unigrams = counter(AbstractString)
    bigrams = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        push!(bigrams[START_MARKER], sent[1])
        for ii in 1:length(sent)-1
            push!(unigrams,sent[ii])
            push!(bigrams[sent[ii]], sent[ii+1])    
        end
        push!(unigrams,sent[end])
        push!(bigrams[sent[end]], END_MARKER)
    end
    [k=>freq2prob(v) for (k,v) in bigrams], freq2prob(unigrams)
end

collect_grams_stats (generic function with 1 method)

In [10]:
#bigram_stat, unigrams = collect_grams_stats(corpus);

In [22]:
using WordEmbeddings

In [34]:
LL, word_indexes, indexed_words = load_word2vec_embeddings("word_emb_data/GoogleNews-vectors-negative300.bin", length(corpus_vocab), corpus_vocab);
@pz LL

LL		Array{Float32,2}	(300,1115)


In [35]:
known_vocab = Set(indexed_words)

Set(AbstractString["carried","diabetic","connections","limousines","anything","loop","fight","r","regular","schedule"  …  "everyday","goes","cost","nearest","further","flare","tell","departs","longer","boeing"])

In [39]:
known_corpus = filter(corpus) do sent
    for word in sent
        if !( word in known_vocab)
            return false
        end
    end
    true
end

4761-element Array{Array{SubString{ASCIIString},1},1}:
 SubString{ASCIIString}["one","way","please"]                                                                                
 SubString{ASCIIString}["on","which","flights","will","lunch","be","served"]                                                 
 SubString{ASCIIString}["which","flights","serve","lunch"]                                                                   
 SubString{ASCIIString}["thank","you"]                                                                                       
 SubString{ASCIIString}["i","live","in","denver"]                                                                            
 SubString{ASCIIString}["round","trip","please"]                                                                             
 SubString{ASCIIString}["one","way"]                                                                                         
 SubString{ASCIIString}["scenario","completed"]                

In [44]:
bigrams, unigrams = collect_grams_stats(known_corpus);

In [46]:
unigrams

Dict{AbstractString,Float64} with 752 entries:
  "limousines"  => 9.556270506163794e-5
  "loop"        => 3.185423502054598e-5
  "null"        => 3.185423502054598e-5
  "regular"     => 3.185423502054598e-5
  "schedule"    => 0.0003185423502054598
  "during"      => 3.185423502054598e-5
  "wondering"   => 3.185423502054598e-5
  "irrelevant"  => 3.185423502054598e-5
  "plan"        => 3.185423502054598e-5
  "those"       => 0.001847545631191667
  "carriers"    => 3.185423502054598e-5
  "rather"      => 6.370847004109197e-5
  "destination" => 3.185423502054598e-5
  "these"       => 0.003058006561972414
  "november"    => 0.00028668811518491386
  "least"       => 0.0009237728155958335
  "us"          => 3.185423502054598e-5
  "either"      => 6.370847004109197e-5
  "francisco"   => 0.003026152326951868
  "scheduled"   => 3.185423502054598e-5
  "possible"    => 0.0001592711751027299
  "response"    => 3.185423502054598e-5
  "fun"         => 3.185423502054598e-5
  "scratch"     => 0.0011467

In [55]:
function select_word{S<:AbstractString,V<:Number}(unigrams::Dict{S,V})
    cutoff = rand()
    total = 0.0
    for next_word in keys(unigrams)
        total+=unigrams[next_word]
        if total>=cutoff
            return next_word
        end
    end
    assert(False, "Should never reach here") 
end

function random_walk(bigrams)
    words=[]
    cur = START_MARKER
    while(cur!=END_MARKER)
        cur = select_word(bigrams[cur])
        push!(words,cur)
    end
    words
    
end

random_walk (generic function with 1 method)

In [67]:
random_walk(bigrams)

13-element Array{Any,1}:
 "i"        
 "will"     
 "do"       
 "you"      
 "know"     
 "the"      
 "coach"    
 "economy"  
 "fares"    
 "for"      
 "san"      
 "francisco"
 "**END**"  